# Machine Learning Project: Data Cleaning

### Question: Can we build a machine learning model that can accurately predict if a borrower will pay off their loan on time or not?

Credit modelling is a well known data science problem that focuses on modeling a borrower's credit risk. Credit has played a key role in the economy for centuries and some form of credit has existed since the beginning of commerce. We'll be working with financial lending data from Lending Club. Lending Club is a marketplace for personal loans that matches borrowers who are seeking a loan with investors looking to lend money and make a return.

Each borrower fills out a comprehensive application, providing their past financial history, the reason for the loan, and more. Lending Club evaluates each borrower's credit score using past historical data (and their own data science process!) and assign an interest rate to the borrower. The interest rate is the percent in addition to the requested loan amount the borrower has to pay back. Lending Club also tries to verify each piece of information the borrower provides but it can't always verify all of the information (usually for regulation reasons).

A higher interest rate means that the borrower is riskier and more unlikely to pay back the loan while a lower interest rate means that the borrower has a good credit history is more likely to pay back the loan. The interest rates range from 5.32% all the way to 30.99% and each borrower is given a grade according to the interest rate they were assigned. If the borrower accepts the interest rate, then the loan is listed on the Lending Club marketplace.

Investors are primarily interested in receiveing a return on their investments. Approved loans are listed on the Lending Club website, where qualified investors can browse recently approved loans, the borrower's credit score, the purpose for the loan, and other information from the application. Once they're ready to back a loan, they select the amount of money they want to fund. Once a loan's requested amount is fully funded, the borrower receives the money they requested minus the origination fee that Lending Club charges.

In [2]:
import pandas as pd
loans_2007 = pd.read_csv('loans_2007.csv')
loans_2007.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [3]:
# After analyzing each column, it was concluded that the following features need to be removed.
drop_columns = ['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d']
loans = loans_2007.drop(drop_columns, axis=1)

In [4]:
# The information isn't available to an investor before the loan is fully funded and 
# it will not be included in the model.
loans = loans.drop(['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp'], 
                   axis=1)

In [5]:
#All of these columns leak data from the future, meaning that they're describing aspects of the loan after 
#it's already been fully funded and started to be paid off by the borrower.

loans = loans.drop(['total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt'],
           axis =1)
loans.shape

(42538, 32)

In [6]:
# 'Loan_status' was chosen as a target column, since it's the only column that directly describes if a loan was paid 
# off on time, had delayed payments, or was defaulted on the borrower. Currently, this column contains text values and
# we need to convert it to a numerical one for training a model. Let's explore the different values in this column and come up with a strategy for converting the values in this column.

loans['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

In [7]:
#Let's remove all the loans that don't contain either "Fully Paid" and "Charged Off" as the loan's status and 
#then transform the "Fully Paid" values to 1 for the positive case and the "Charged Off" values to 0 for 
#the negative case.

status_dict ={
    'loan_status': {
    'Fully Paid': 1,
    'Charged Off': 0
    }
}
loans = loans.replace(status_dict) 
loans = loans[(loans['loan_status']==1) | (loans['loan_status']==0)]

In [9]:
#let's look for any columns that contain only one unique value and remove them. 
#These columns won't be useful for the model since they don't add any information to each loan application. 
drop_columns = []
for i in loans.columns:
    col_series = loans[i].dropna()
    if len(col_series.unique()) == 1:
        drop_columns.append(i)
loans = loans.drop(drop_columns, axis=1)


# Machine Learning Project: Preparing the features

We'll prepare the data for machine learning by focusing on handling missing values, converting categorical columns to numeric columns, and removing any other extraneous columns we encounter throughout this process.

This is because the mathematics underlying most machine learning models assumes that the data is numerical and contains no missing values. To reinforce this requirement, scikit-learn will return an error if you try to train a model using data that contain missing values or non-numeric values when working with models like linear regression and logistic regression.

In [10]:
loans.isnull().sum()

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

In [11]:
#Let's use the strategy of removing the pub_rec_bankruptcies column first, since nearly 94% of values are in the same 
#category. Then, we removed all rows containing any missing values at all to cover both of these cases. This way, we 
#only remove the rows containing missing values for the "emp_length", title and revol_util columns, but not the 
#pub_rec_bankruptcies column.

loans = loans.drop('pub_rec_bankruptcies', axis=1)
loans = loans.dropna()
print(loans.dtypes.value_counts())

object     12
float64    10
dtype: int64


In [12]:
#While the numerical columns can be used natively with scikit-learn, the object columns that contain text need to be 
#converted to numerical data types. Let's return a new Dataframe containing just the object columns so we can explore 
#them in more depth.

object_columns = loans.select_dtypes(include=['object'])
print(object_columns.iloc[0])

term                     36 months
int_rate                    10.65%
emp_length               10+ years
home_ownership                RENT
verification_status       Verified
loan_status                      1
purpose                credit_card
title                     Computer
addr_state                      AZ
earliest_cr_line          Jan-1985
revol_util                   83.7%
last_credit_pull_d        Jun-2016
Name: 0, dtype: object


In [13]:
#Let's explore the unique value counts of the columnns that seem like they contain categorical values.

cols = ['home_ownership', 'verification_status', 'emp_length', 'term', 'addr_state']
for i in cols:
    print(loans[i].value_counts())

RENT        18112
MORTGAGE    16686
OWN          2778
OTHER          96
NONE            3
Name: home_ownership, dtype: int64
Not Verified       16281
Verified           11856
Source Verified     9538
Name: verification_status, dtype: int64
10+ years    8545
< 1 year     4513
2 years      4303
3 years      4022
4 years      3353
5 years      3202
1 year       3176
6 years      2177
7 years      1714
8 years      1442
9 years      1228
Name: emp_length, dtype: int64
 36 months    28234
 60 months     9441
Name: term, dtype: int64
CA    6776
NY    3614
FL    2704
TX    2613
NJ    1776
IL    1447
PA    1442
VA    1347
GA    1323
MA    1272
OH    1149
MD    1008
AZ     807
WA     788
CO     748
NC     729
CT     711
MI     678
MO     648
MN     581
NV     466
SC     454
WI     427
OR     422
AL     420
LA     420
KY     311
OK     285
KS     249
UT     249
AR     229
DC     209
RI     194
NM     180
WV     164
HI     162
NH     157
DE     110
MT      77
WY      76
AK      76
SD      60
VT  

In [14]:
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                                                  2068
Debt Consolidation Loan                                             1599
Personal Loan                                                        624
Consolidation                                                        488
debt consolidation                                                   466
Credit Card Consolidation                                            345
Home Improvement                                                     336
Debt consolidation                                                   314
Small Business Loan                                                  298
Credit Card Loan                                                     294
Personal                                                             290
Consolidation Loan                                                   250
Home Improvement Loan                                                228
personal loan                                      

In [15]:
#let's look at the unique value counts for the purpose and title columns to understand which column we want to keep.
print(loans["title"].value_counts())
print(loans["purpose"].value_counts())

Debt Consolidation                                                  2068
Debt Consolidation Loan                                             1599
Personal Loan                                                        624
Consolidation                                                        488
debt consolidation                                                   466
Credit Card Consolidation                                            345
Home Improvement                                                     336
Debt consolidation                                                   314
Small Business Loan                                                  298
Credit Card Loan                                                     294
Personal                                                             290
Consolidation Loan                                                   250
Home Improvement Loan                                                228
personal loan                                      

In [16]:
#We erred on the side of being conservative with the 10+ years, < 1 year and n/a mappings. We assume that people 
#who may have been working more than 10 years have only really worked for 10 years. We also assume that people 
#who've worked less than a year or if the information is not available that they've worked for 0. This is a general
#heuristic but it's not perfect.

loans['int_rate'] = loans['int_rate'].str.replace('%',"").astype("float")
loans["revol_util"] = loans["revol_util"].str.replace('%',"").astype("float")
loans['loan_status'] = loans['loan_status'].astype("float")


mapping = {'emp_length' : {
   "10+ years": 10,
    "< 1 year": 0,
    "2 years": 2,
    "3 years": 3,
    "4 years": 4,
    "5 years": 5,
    "1 year": 1,
    "6 years": 6,
    "7 years": 7,
    "8 years": 8,
    "9 years": 9,
    "n/a": 0
}}

loans = loans.replace(mapping)
loans['emp_length'] = loans['emp_length'].astype("float")
loans = loans.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1)

In [17]:
#Let's now encode the "home_ownership", "verification_status", "purpose", and term columns as dummy variables 
#so we can use them in our model.
loans = pd.get_dummies(loans, columns =['home_ownership', 'verification_status', 'purpose', 'term'])


# Machine Learning Project: Making Predictions

#### We noticed that there's a class imbalance in our target column, "loan_status". There are about 6 times as many loans that were paid off on time (positive case, label of 1) than those that weren't (negative case, label of 0). Imbalances can cause issues with many machine learning algorithms, where they appear to have high accuracy, but actually aren't learning from the training data. Because of its potential to cause issues, we need to keep the class imbalance in mind as we build machine learning models.


In [18]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

cols = list(loans.columns)
cols.remove("loan_status")
features = loans[cols]
target = loans['loan_status']
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates
tpr = tp  / (tp + fn)
fpr = fp  / (fp + tn)
print(tpr)
print(fpr)

/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [19]:
#As you can see from the last screen, our fpr and tpr are around what we'd expect if the model was predicting
#all ones.We can do this by setting the class_weight parameter to "balanced" when creating the LogisticRegression 
#instance. This tells scikit-learn to penalize the misclassification of the minority class during the training process. 
#The penalty means that the logistic regression classifier pays more attention to correctly classifying rows where 
# "loan_status" is 0. This lowers accuracy when loan_status is 1, but raises accuracy when loan_status is 0.

#By setting the class_weight parameter to balanced, the penalty is set to be inversely proportional to the class
#frequencies. This would mean that for the classifier, correctly classifying a row where loan_status is 0 is 6 times
#more important than correctly classifying a row where loan_status is 1.

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_predict
lr = LogisticRegression(class_weight='balanced')
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
# Rates

tpr = tp / (tp+fn)
fpr = fp / (fp+tn)
print(tpr)
print(fpr)

/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


0.6263109596224437
0.611253701875617


In [20]:
#We significantly improved false positive rate in the last screen by balancing the classes, which reduced true 
#positive rate. Our true positive rate is now around 63%, and our false positive rate is around 61%. From a 
#conservative investor's standpoint, it's reassuring that the false positive rate is lower because it means 
#that we'll be able to do a better job at avoiding bad loans than if we funded everything. However, we'd only ever 
#decide to fund 63% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

#We can also specify a penalty manually if we want to adjust the rates more. To do this, we need to pass in a 
#dictionary of penalty values to the class_weight parameter

penalty = {
    0: 10,
    1: 1
}
lr = LogisticRegression(class_weight=penalty)

predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)
predictions = cross_val_predict(lr, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])
tpr = tp / (tp+fn)
fpr = fp / (fp+tn)

print(tpr)
print(fpr)

/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Defau

0.22787755637126378
0.2252714708785785


In [21]:
#It looks like assigning manual penalties lowered the false positive rate to 22.5%, and thus lowered our risk.
#Note that this comes at the expense of true positive rate. While we have fewer false positives, we're also missing 
#opportunities to fund more loans and potentially make more money. Given that we're approaching this as a conservative 
#investor, this strategy makes sense, but it's worth keeping in mind the tradeoffs

#Let's try a more complex algorithm, random forest. Random forests are able to work with nonlinear data, and learn 
#complex conditionals.  Training a random forest algorithm may enable us to get more accuracy due to columns that 
#correlate nonlinearly with loan_status.

from sklearn.ensemble import RandomForestClassifier

rf = RandomForestClassifier(class_weight="balanced", random_state=1)
predictions = cross_val_predict(rf, features, target, cv=3)
predictions = pd.Series(predictions)

# False positives.
fp_filter = (predictions == 1) & (loans["loan_status"] == 0)
fp = len(predictions[fp_filter])

# True positives.`
tp_filter = (predictions == 1) & (loans["loan_status"] == 1)
tp = len(predictions[tp_filter])

# False negatives.
fn_filter = (predictions == 0) & (loans["loan_status"] == 1)
fn = len(predictions[fn_filter])

# True negatives
tn_filter = (predictions == 0) & (loans["loan_status"] == 0)
tn = len(predictions[tn_filter])

# Rates
tpr = tp / (tp + fn)
fpr = fp / (fp + tn)

print(tpr)
print(fpr)



/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
/Users/aassamidanov/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


0.9630637126376508
0.9636722606120435


#### Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.
#### Ultimately, our best model had a false positive rate of 22.5%, and a true positive rate of 22.8%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 22.5% of borrowers defaulting, and that the pool of 22.8% of borrowers is large enough to make enough interest money to offset the losses.